In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
DATA_VERSION = "v003"
TRIAL_NO = "006"

In [24]:
importance_list = []
for i in range(8):
    path = f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_{i}.csv"
    importance_df = pd.read_csv(path, index_col=0)
    importance_df = pd.concat([g.set_index("feature")[["importance"]].rename({"importance":f"fold_{i}"}, axis=1) for i, g in importance_df.groupby("fold")],axis=1)
    importance_df["ave"] = importance_df.mean(axis=1)
    importance_df["ratio"] = importance_df["ave"] / importance_df["ave"].sum()
    importance_df.sort_values("ratio", ascending=False ,inplace=True)
    importance_list += [importance_df]
    #break

In [30]:
importance_all_df = pd.concat([importance_list[i]["ratio"] for i in range(8)], axis=1)
importance_all_df.columns = [f"type_{i}" for i in range(8)]
importance_all_df["ave"] = importance_all_df.mean(axis=1)
importance_all_df.sort_values("ave", ascending=False ,inplace=True)
importance_all_df.to_excel(f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_all.xlsx")
importance_all_df

/Users/kenichi.matsui/anaconda3/envs/kaggle_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,ave
oof_fc,0.02728,0.03012,0.03045,0.02300,0.02914,0.03484,0.02359,0.03607,0.02931
f006:dist_origin_mean,0.02339,0.03356,0.01760,0.02562,0.02572,0.01985,0.02210,0.02770,0.02444
abs_dist,0.02477,0.02901,0.01974,0.02290,0.02339,0.01688,0.01834,0.01880,0.02173
f006:dist_from_origin_0,0.02242,0.02339,0.02076,0.02314,0.02206,0.02005,0.01953,0.02043,0.02147
molecule_dist_mean,0.01950,0.02686,0.01429,0.02192,0.01932,0.01590,0.01978,0.02024,0.01973
molecule_dist_max,0.01931,0.02763,0.01499,0.02172,0.02121,0.01157,0.01932,0.01998,0.01947
f006:dist_from_origin_1,0.01822,0.01879,0.01659,0.02410,0.01751,0.01659,0.02308,0.01660,0.01894
molecule_atom_1_dist_mean,0.02010,0.02356,0.01505,0.02120,0.01759,0.01544,0.02065,0.01677,0.01880
molecule_atom_index_0_x_1_std,0.01975,0.02042,0.01684,0.01806,0.01885,0.01644,0.01678,0.01748,0.01808
z_0,0.01787,0.01841,0.01613,0.01697,0.02096,0.01608,0.01678,0.02037,0.01795
